#deconvolution - lymphoid visualiztion

In [ ]:
library(Seurat)
library(ggplot2)
library(patchwork)
library(dplyr)

In [ ]:
object <- readRDS("./RDS/250520.combined_3.4_visium_label_update_hires.rds")

In [ ]:
object

In [ ]:
object <- subset(object, idents = c("Neutrophilic pustule", "Eccrine gland coil", "Adipocyte", "Eccrine gland duct"), invert = TRUE)


In [ ]:
ref <- readRDS("/data/User/revolvefire/projects/3.PPP_231118/231128_PPP/RDS/250416_integrated_total_rawcounts.rds")

In [ ]:
ref <- NormalizeData(ref)

In [ ]:
ref <- FindVariableFeatures(ref)

In [ ]:
Idents(ref) <- "label2"

In [ ]:
# the annotation is stored in the 'subclass' column of object metadata
DimPlot(ref, group.by = "label2", label = TRUE)

In [ ]:
ref <- subset(ref, idents = c("Melanocyte"), invert = TRUE)

In [ ]:
# the annotation is stored in the 'subclass' column of object metadata
DimPlot(ref, group.by = "label3", label = TRUE)

In [ ]:
ref@meta.data$label3 <- as.character(ref@meta.data$label3)
ref@meta.data$label3[ref@meta.data$label3 %in% c("BK-1", "BK-2", "BK-3")] <- "BK-1-3"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-1", "SBK-2")] <- "SBK-1-2"
ref@meta.data$label3[ref@meta.data$label3 %in% c("SBK-3", "SBK-4")] <- "SBK-3-4"

In [ ]:
ref@meta.data$label3[ref@meta.data$label3 %in% c("Superficial FIB", "FRC-like FIB", "Inf universal FIB", "Universal FIB")] <- "Fibroblast"

In [ ]:
anchors <- FindTransferAnchors(reference = ref, query = object, normalization.method="LogNormalize")
predictions.assay <- TransferData(anchorset = anchors, refdata = ref$label3, prediction.assay = TRUE,
    weight.reduction = object[["pca"]], dims = 1:30)
object[["predictions"]] <- predictions.assay

In [ ]:
row_order <- c("BK-1-3", "SBK-1-2", "SBK-3-4", "SBK-5", "Th17", "CD4em", "CD4n", "CD8/NK", "pDC", "mDC", "cDC2-2", "cDC2-1", "Macrophage", "Fibroblast", "Endothelial", "Pericyte")

In [ ]:
col_order <- c('Basal KC','Differentiating KC','Terminally diff KC','Pustular KC', 'Lymphoid-CCL19+ niche','pDC','a/mDC', 'Inflammatory Fib/Mac','Fibroblast', 'Endo/Peri')

In [ ]:
data_matrix <- object@assays[["predictions"]]@data
data_matrix <- data_matrix[rownames(data_matrix) != "max", ]

cell_types <- object$label
cell_ids <- colnames(data_matrix)
cell_ids_by_type <- split(cell_ids, cell_types[cell_ids])

sum_values <- matrix(NA, nrow = nrow(data_matrix), ncol = length(unique(cell_types)))
colnames(sum_values) <- unique(cell_types)
for (cell_type in unique(cell_types)) {
  selected_columns <- cell_ids_by_type[[cell_type]]
  sum_values[, cell_type] <- rowSums(data_matrix[, selected_columns, drop = FALSE])
}
rownames(sum_values) <- rownames(data_matrix)

min_max_standarize <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}

data <- t(apply(sum_values, 1, min_max_standarize))
data_df <- as.data.frame(data)
colnames(data_df) <- colnames(sum_values)
rownames(data_df) <- rownames(sum_values)

In [ ]:
colnames(sum_values)

In [ ]:
rownames(sum_values)

In [ ]:
data_df <- data_df[row_order, col_order, drop=FALSE]

In [ ]:
library(pheatmap)
library(viridis)

In [ ]:
pheatmap(t(data_df), 
                   color = plasma(300), 
                   cluster_rows = FALSE, 
                   cluster_cols = FALSE,
                   show_rownames = TRUE,          
                   show_colnames = TRUE,
                   fontsize = 15,   
                   angle_col = "90")

In [ ]:
# Subset for slide1
s1 <- subset(object, subset = slide == "slide1")

# Subset for slide2
s2 <- subset(object, subset = slide == "slide2")

In [ ]:
library(RColorBrewer)

In [ ]:
DefaultAssay(s1) <- "predictions"
immune_labels <- c("Th17", "CD4em", "CD4n", "CD8/NK", "pDC", "cDC2-1", "cDC2-2", "mDC",  "Fibroblast")
niche_cells <- WhichCells(s1, expression = label == "Lymphoid-CCL19+ niche")
scores <- FetchData(s1, vars = immune_labels, layer = "data")[niche_cells, ]
thresholds <- apply(scores, 2, quantile, probs = 0.8, na.rm = TRUE)

for (ct in immune_labels) {
  scores[[ct]][scores[[ct]] < thresholds[ct]] <- NA
}

max_labels <- apply(scores, 1, function(x) {
  if (all(is.na(x))) return(NA)
  colnames(scores)[which.max(x)]
})
max_scores <- apply(scores, 1, function(x) ifelse(all(is.na(x)), NA, max(x, na.rm = TRUE)))


s1$top_label <- NA
s1$score <- NA
s1$top_label[niche_cells] <- max_labels
s1$score[niche_cells] <- max_scores

s1$top_label <- factor(s1$top_label, levels = immune_labels)

palette <- c(
  "Th17"       = "#EF476F",
  "CD4em"      = "#fb8500",
  "CD4n"       = "#ffd60a",
  "CD8/NK"     = "#91cb3e",
  "pDC"        = "#20bf55",
  "cDC2-1"     = "#00b4d8",
  "cDC2-2"     = "#0077b6",
  "mDC"        = "#8f03ff",
  "Fibroblast" = "#a4b0b4"
)

In [ ]:
p <- SpatialPlot(
  object = subset(s1, cells = niche_cells),
  group.by = "top_label",
  pt.size.factor = 2.5,
  image.scale = "hires",
  image.alpha = 0.2,
  cols = palette
) +
 
  theme(
    legend.text = element_text(size = 12),
    legend.title = element_text(size = 14),
    legend.key.size = unit(1.5, "lines")
  ) +
  guides(
    fill = guide_legend(override.aes = list(size = 6))  # larger dots in legend
  )

# Show the plot
print(p)

In [ ]:
rotate_image <- function(p, rot_angle) {
  gt <- ggplot_gtable(ggplot_build(p))
  panel_idx <- which(gt$layout$name == "panel")
  rot_vp <- viewport(angle = rot_angle)
  gt[["grobs"]][[panel_idx]] <- editGrob(gt[["grobs"]][[panel_idx]], vp = rot_vp)
  p_rot <- ggdraw() + draw_grob(gt)
  
  return(p_rot)
}

In [ ]:
library(cowplot)
library(ggplot2)
library(grid)

In [ ]:
p_rot <- rotate_image(p, 270)

In [ ]:
p_rot